In [1]:
# %tensorflow_version 1.x
import pandas as pd
import numpy as np
np.random.seed(1337)
from keras import Sequential
from keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub

Using TensorFlow backend.


In [2]:
tf.test.is_gpu_available()

True

In [3]:
LABEL = 'label3-g'

In [4]:
transcript_raw = pd.read_csv('Data/transcript.csv')
transcript_raw.columns = transcript_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
transcript_raw = transcript_raw[transcript_raw.google_transcript.notnull()]
transcript_raw.head()

,unnamed:_0,interview_id,interview_date,record_creation_date,venue,address_name,street,city,state_/_province,postal_code,...,interviewer_relationship_5_to_storyteller_2,language_1,language_2,language_3,keywords_-_fixed_subjects,keywords_-_general,keywords_-_places,deep_speech,human_transcript,google_transcript
0,0,MBY005668,9/5/09 11:30,9/5/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,NaN,Spanish,NaN,NaN,Achievements and Awards\n\nChildren\n\nCommuni...,"bilingual\n\nBilingue\n\nChihuahua, Mexico\n\n...",,i am an amadis so my lawyer in whitewashin or ...,NaN,I mean I'm already so my lawyer Singapore and ...
1,1,MBY005704,9/12/09 14:30,9/12/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,NaN,Spanish,NaN,NaN,Childcare\n\nChildren\n\nTraumatic Memories,accidents\n\nbilingual\n\nbirth of first child...,NaN,all me i olliphant elocution cutaneous or a sa...,NaN,call mi amor Alejandro De La Cruz Munoz arrest...
2,2,MBY005712,9/13/09 17:30,9/13/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,NaN,Spanish,NaN,NaN,Anniversaries\n\nChildren\n\nCommunity Busines...,accidents\n\nbaile\n\ncar accident\n\ncemetery...,NaN,it kanonsionni neesnetasis officiousness miste...,NaN,okay mean Hombres es el Amante Samira cincuent...
3,3,MBY005713,9/14/09 9:30,9/14/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,NaN,English,NaN,NaN,Birth\n\nBurials\n\nCancer\n\nChildcare\n\nChi...,alcohol\n\nbirth of first child\n\nchemotherap...,NaN,my name is maria humphry i'm twenty six to day...,NaN,my name is Melissa Humphrey I'm 26 today's dat...
4,4,MBY005730,9/18/09 12:30,9/18/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,NaN,NaN,NaN,NaN,Changes In Education\n\nCommunity Businesses\n...,anecdotes (humorous but true stories)\n\nappea...,NaN,the woman silence hello my name is alan fucara...,NaN,hello my name is Adolfo Carranza Junior and I'...


In [5]:
transcript_raw.shape

(349, 61)

In [6]:
label_raw = pd.read_csv('Data/labels.csv')
label_raw.columns = label_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
label_raw.head()

,interview_id,interview_date,record_creation_date,venue,address_name,street,city,state_/_province,postal_code,interview_description,...,interviewer_relationship_1_to_storyteller_1,language_1,keywords_-_fixed_subjects,keywords_-_general,label1-fs,label2-fs,label3-fs,label1-g,label2-g,label3-g
0,MBY005668,2009/9/5 11:30,2009/9/5,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Rita Elvia Loya (38) habla con su esposo Omar ...,...,wife,Spanish,Achievements and Awards\n\nChildren\n\nCommuni...,"bilingual\n\nBilingue\n\nChihuahua, Mexico\n\n...",Children,Workday Life,Immigration Stories,social beliefs and practices,ethnicity,memories of former times
1,MBY005712,2009/9/13 17:30,2009/9/13,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Cecilio Montes (46) habla con esposa Bertha Ja...,...,esposa,Spanish,Anniversaries\n\nChildren\n\nCommunity Busines...,accidents\n\nbaile\n\ncar accident\n\ncemetery...,Children,Workday Life,Immigration Stories,memories of former times,spouse,family tradition
2,MBY005713,2009/9/14 9:30,2009/9/14,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Alice talks about childhood. Melissa remember...,...,grandmother,English,Birth\n\nBurials\n\nCancer\n\nChildcare\n\nChi...,alcohol\n\nbirth of first child\n\nchemotherap...,Parents,Children,Marriage,memories of growing up,memories of former times,family tradition
3,MBY005730,2009/9/18 12:30,2009/9/18,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Adolph Carranza Jr. (64) talks with friend Ele...,...,friend,NaN,Changes In Education\n\nCommunity Businesses\n...,anecdotes (humorous but true stories)\n\nappea...,Parents,Schools,Workday Life,memories of growing up,social beliefs and practices,ethnicity
4,MBY005764,2009/9/28 10:30,2009/9/28,MobileBooth West (MBY),MobileBooth West,Public Library,Colorado Springs,CO,80911,Rosana interviews her friend Alberta about her...,...,friend,NaN,Education\n\nAchievements and Awards\n\nArmy\n...,advocacy\n\nadvocate\n\ncollege\n\ndevelopment...,Schools,Children,Workday Life,memories of former times,school day memories,spouse


In [7]:
train_raw = transcript_raw.merge(label_raw, on='interview_id')
train_raw.head()

,unnamed:_0,interview_id,interview_date_x,record_creation_date_x,venue_x,address_name_x,street_x,city_x,state_/_province_x,postal_code_x,...,interviewer_relationship_1_to_storyteller_1_y,language_1_y,keywords_-_fixed_subjects_y,keywords_-_general_y,label1-fs,label2-fs,label3-fs,label1-g,label2-g,label3-g
0,0,MBY005668,9/5/09 11:30,9/5/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,wife,Spanish,Achievements and Awards\n\nChildren\n\nCommuni...,"bilingual\n\nBilingue\n\nChihuahua, Mexico\n\n...",Children,Workday Life,Immigration Stories,social beliefs and practices,ethnicity,memories of former times
1,2,MBY005712,9/13/09 17:30,9/13/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,esposa,Spanish,Anniversaries\n\nChildren\n\nCommunity Busines...,accidents\n\nbaile\n\ncar accident\n\ncemetery...,Children,Workday Life,Immigration Stories,memories of former times,spouse,family tradition
2,3,MBY005713,9/14/09 9:30,9/14/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,grandmother,English,Birth\n\nBurials\n\nCancer\n\nChildcare\n\nChi...,alcohol\n\nbirth of first child\n\nchemotherap...,Parents,Children,Marriage,memories of growing up,memories of former times,family tradition
3,4,MBY005730,9/18/09 12:30,9/18/09,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428.0,...,friend,NaN,Changes In Education\n\nCommunity Businesses\n...,anecdotes (humorous but true stories)\n\nappea...,Parents,Schools,Workday Life,memories of growing up,social beliefs and practices,ethnicity
4,6,MBY005764,9/28/09 10:30,9/28/09,MobileBooth West (MBY),MobileBooth West,Public Library,Colorado Springs,CO,80911.0,...,friend,NaN,Education\n\nAchievements and Awards\n\nArmy\n...,advocacy\n\nadvocate\n\ncollege\n\ndevelopment...,Schools,Children,Workday Life,memories of former times,school day memories,spouse


In [8]:
train_raw.shape

(332, 84)

In [9]:
train_raw = train_raw[['google_transcript', LABEL]]
train_raw.reset_index(inplace=True, drop=True)
train_raw.head()

,google_transcript,label3-g
0,I mean I'm already so my lawyer Singapore and ...,memories of former times
1,okay mean Hombres es el Amante Samira cincuent...,family tradition
2,my name is Melissa Humphrey I'm 26 today's dat...,family tradition
3,hello my name is Adolfo Carranza Junior and I'...,ethnicity
4,my number is rational and pony my name is Rose...,spouse


In [10]:
for l in np.unique(train_raw[LABEL]):
  print(l)

anecdotes (humorous but true stories)
college
economic beliefs and practices
ethnicity
family characters
family naming and nicknames
family tradition
memories of former times
political beliefs and practices
religious beliefs and practices
school day memories
spouse


In [11]:
train_raw=train_raw.rename(columns = {'google_transcript':'text', LABEL:'label'})
train_raw.head()

,text,label
0,I mean I'm already so my lawyer Singapore and ...,memories of former times
1,okay mean Hombres es el Amante Samira cincuent...,family tradition
2,my name is Melissa Humphrey I'm 26 today's dat...,family tradition
3,hello my name is Adolfo Carranza Junior and I'...,ethnicity
4,my number is rational and pony my name is Rose...,spouse


In [12]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
train_raw['label'] = LE.fit_transform(train_raw['label'])
train_raw.head()

,text,label
0,I mean I'm already so my lawyer Singapore and ...,7
1,okay mean Hombres es el Amante Samira cincuent...,6
2,my name is Melissa Humphrey I'm 26 today's dat...,6
3,hello my name is Adolfo Carranza Junior and I'...,3
4,my number is rational and pony my name is Rose...,11


In [13]:
num_labels = len(np.unique(train_raw['label']))
num_labels

12

In [14]:
train_r = train_raw.copy()

In [15]:
train_r = train_r.reindex(np.random.permutation(train_r.index))
train_r.head()

,text,label
260,my name is Samantha Garcia in 21 today's date ...,7
90,but it was hilarious I have 24 years old today...,3
163,okay my name is Juan Carlos Padilla I'm 34 yea...,3
188,my name is Lisa Padilla I'm 47 today is March ...,3
53,Santa Monica Garcia then go cuarenta siete ano...,0


In [16]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [17]:
train_r['text']  = train_r.text.apply(clean_txt)
train_r.head()

,text,label
260,my name is Samantha Garcia in 21 todays date i...,7
90,but it was hilarious I have 24 years old today...,3
163,okay my name is Juan Carlos Padilla Im 34 year...,3
188,my name is Lisa Padilla Im 47 today is March 1...,3
53,Santa Monica Garcia then go cuarenta siete ano...,0


In [18]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_r, test_size=0.2)
while(len(np.unique(train.label)) != num_labels):
      train, val = train_test_split(train_r, test_size=0.2)
train.head()

,text,label
79,Douglas Sanchez 18 January 16th 2010 Yuma Ariz...,3
17,hi my name is Heidi Christine Johnson Im 38 ye...,7
75,hello my name is Catalina Correa age and my th...,3
237,hello my name is Ernesto Cassidys Im 25 years ...,3
263,and you know Melissa Lopez Chavez single cincu...,3


In [19]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,text,label
0,Douglas Sanchez 18 January 16th 2010 Yuma Ariz...,3
1,hi my name is Heidi Christine Johnson Im 38 ye...,7


In [20]:
val.reset_index(drop=True, inplace=True)
val.head(2)

,text,label
0,me nombre zombie Salcedo unfortunately it stay...,3
1,my name is Erin Selena Reyes my age is actuall...,3


In [21]:
val.shape, train.shape

((67, 2), (265, 2))

In [22]:
#Installing BERT module
!pip install bert-tensorflow

In [23]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [24]:
# Set the output directory for saving model file
OUTPUT_DIR = './bert_output/'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: ./bert_output/ *****


In [25]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
# print("Test Set Shape :", test.shape)

Training Set Shape : (265, 2)
Validation Set Shape : (67, 2)


In [26]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(train.label)]
label_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [27]:
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [28]:
train['text_split'] = train[DATA_COLUMN].apply(get_split)
train.head()

C:\Users\ej\anaconda3\envs\bert\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,label,text_split
0,Douglas Sanchez 18 January 16th 2010 Yuma Ariz...,3,[Douglas Sanchez 18 January 16th 2010 Yuma Ari...
1,hi my name is Heidi Christine Johnson Im 38 ye...,7,[hi my name is Heidi Christine Johnson Im 38 y...
2,hello my name is Catalina Correa age and my th...,3,[hello my name is Catalina Correa age and my t...
3,hello my name is Ernesto Cassidys Im 25 years ...,3,[hello my name is Ernesto Cassidys Im 25 years...
4,and you know Melissa Lopez Chavez single cincu...,3,[and you know Melissa Lopez Chavez single cinc...


In [29]:
val['text_split'] = val[DATA_COLUMN].apply(get_split)
val.head(2)

C:\Users\ej\anaconda3\envs\bert\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,label,text_split
0,me nombre zombie Salcedo unfortunately it stay...,3,[me nombre zombie Salcedo unfortunately it sta...
1,my name is Erin Selena Reyes my age is actuall...,3,[my name is Erin Selena Reyes my age is actual...


In [30]:
train_l = []
label_l = []
index_l =[]
for idx,row in train.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['label'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(7756, 7756, 7756)

In [31]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['label'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(2075, 2075, 2075)

In [32]:
train_df = pd.DataFrame({DATA_COLUMN:train_l, LABEL_COLUMN:label_l})
train_df.head()

,text,label
0,Douglas Sanchez 18 January 16th 2010 Yuma Ariz...,3
1,that that particular time contrary to now ther...,3
2,bar called The Cameo and at that time traffic ...,3
3,was one of the very first projects I ever did ...,3
4,in and it was restricted and I started bring I...,3


In [33]:
val_df = pd.DataFrame({DATA_COLUMN:val_l, LABEL_COLUMN:val_label_l})
val_df.head()

,text,label
0,me nombre zombie Salcedo unfortunately it stay...,3
1,can be left alone in Spanish books what Im int...,3
2,to you yet cast device to spend all 1 sueno de...,3
3,TransCanada Latinas much has been said when yo...,3
4,CC C People are getting a list A to Z Pinot co...,3


In [34]:
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [35]:
train_InputExamples

0       <bert.run_classifier.InputExample object at 0x...
1       <bert.run_classifier.InputExample object at 0x...
2       <bert.run_classifier.InputExample object at 0x...
3       <bert.run_classifier.InputExample object at 0x...
4       <bert.run_classifier.InputExample object at 0x...
                              ...                        
7751    <bert.run_classifier.InputExample object at 0x...
7752    <bert.run_classifier.InputExample object at 0x...
7753    <bert.run_classifier.InputExample object at 0x...
7754    <bert.run_classifier.InputExample object at 0x...
7755    <bert.run_classifier.InputExample object at 0x...
Length: 7756, dtype: object

In [36]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  Douglas Sanchez 18 January 16th 2010 Yuma Arizona Michael is my friend Michael H Bachman Im 67 January 16th 2010 in Yuma Arizona my relationship is a friend and mentor Im so Michael what brought you to Yuma and when you got here or just describe the LGBT community when my boyfriend of 17 years died in the car crash in Fresno California made a decision then I would go and it was simultaneous that I would go down to Yuma come over to Yuma because my father was also ill and he was about to die he we had about six months so I decided to just take a year and come down here and stay with him and in that six months time in Yuma got involved immediately with the community the Yuma Fine Arts the again whatever was going on I just got involved in it and that that particular time contrary to now there was a tremendously robots the gay community which I found appealing because every cuz it was congenial it was neigh

In [37]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [38]:
len(tokenizer.vocab.keys())

30522

In [39]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['douglas', 'sanchez', '18', 'january', '16th', '2010', 'yu', '##ma', 'arizona', 'michael', 'is', 'my', 'friend', 'michael', 'h', 'bach', '##man', 'im', '67', 'january', '16th', '2010', 'in', 'yu', '##ma', 'arizona', 'my', 'relationship', 'is', 'a', 'friend', 'and', 'mentor', 'im', 'so', 'michael', 'what', 'brought', 'you', 'to', 'yu', '##ma', 'and', 'when', 'you', 'got', 'here', 'or', 'just', 'describe', 'the', 'lgbt', 'community', 'when', 'my', 'boyfriend', 'of', '17', 'years', 'died', 'in', 'the', 'car', 'crash', 'in', 'fresno', 'california', 'made', 'a', 'decision', 'then', 'i', 'would', 'go', 'and', 'it', 'was', 'simultaneous', 'that', 'i', 'would', 'go', 'down', 'to', 'yu', '##ma', 'come', 'over', 'to', 'yu', '##ma', 'because', 'my', 'father', 'was', 'also', 'ill', 'and', 'he', 'was', 'about', 'to', 'die', 'he', 'we', 'had', 'about', 'six', 'months', 'so', 'i', 'decided', 'to', 'just', 'take', 'a', 'year', 'and', 'come', 'down', 'here', 'and', 'stay', 'with', 'him', 'and', 'in', 

In [40]:
MAX_SEQ_LENGTH = 200

In [41]:
# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 7756


INFO:tensorflow:Writing example 0 of 7756


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 3 (id = 3)


INFO:tensorflow:label: 3 (id = 3)


In [42]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  # with tf.Session() as sess:
  output_layer1 = bert_outputs["pooled_output"]
  # output_layer1 = 999
  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.8)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs, output_layer1)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [43]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs, output_layer) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels,
          'pooled_output': output_layer
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [44]:
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [45]:
num_train_steps, len(label_list)

(969, 12)

In [46]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': './bert_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021151129F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './bert_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021151129F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [48]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 969 into ./bert_output/model.ckpt.


INFO:tensorflow:Saving checkpoints for 969 into ./bert_output/model.ckpt.


INFO:tensorflow:Loss for final step: 1.4718494.


INFO:tensorflow:Loss for final step: 1.4718494.


Training took time  0:07:16.634450


In [49]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\Users\ej\anaconda3\envs\bert\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 969: eval_accuracy = 0.62168676, false_negatives = 4.0, false_positives = 261.0, global_step = 969, loss = 1.4183246, true_negatives = 2.0, true_positives = 1808.0


INFO:tensorflow:Saving dict for global step 969: eval_accuracy = 0.62168676, false_negatives = 4.0, false_positives = 261.0, global_step = 969, loss = 1.4183246, true_negatives = 2.0, true_positives = 1808.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 969: ./bert_output/model.ckpt-969


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 969: ./bert_output/model.ckpt-969


{'eval_accuracy': 0.62168676,
 'false_negatives': 4.0,
 'false_positives': 261.0,
 'loss': 1.4183246,
 'true_negatives': 2.0,
 'true_positives': 1808.0,
 'global_step': 969}

In [50]:
# A method to get predictions
def getPrediction(in_sentences, type_output = "features"):
  #A list to map the actual labels to the predictions
  labels = np.unique(train['label'])
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  if type_output == "features":
    return [prediction['pooled_output'] for _,prediction in enumerate(predictions) ]
  else:
    return ([(sentence, prediction['probabilities'],
              prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)])

In [51]:
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
MAX_SEQ_LENGTH

200

In [52]:
train_df.shape, val_df.shape

((7756, 2), (2075, 2))

In [53]:
# transcript_raw = transcript_raw[transcript_raw.human_transcript.notnull()]
# train_df = train_df[train_df.DATA_COLUMN.notnull()]
# train_df.shape
train_df.head()

,text,label
0,Douglas Sanchez 18 January 16th 2010 Yuma Ariz...,3
1,that that particular time contrary to now ther...,3
2,bar called The Cameo and at that time traffic ...,3
3,was one of the very first projects I ever did ...,3
4,in and it was restricted and I started bring I...,3


In [54]:
%%time
tr_emb = np.apply_along_axis(getPrediction, 0,np.array(train_df[DATA_COLUMN]))

Wall time: 1min 41s


In [55]:
%%time
val_emb = np.apply_along_axis(getPrediction, 0,np.array(val_df[DATA_COLUMN]))
val_emb.shape

Wall time: 32.5 s


(2075, 768)

In [56]:
val_emb.shape, tr_emb.shape

((2075, 768), (7756, 768))

In [57]:
aux = -1
len_l = 0
train_x = {}
for l, emb in zip(index_l, tr_emb):
  if l in train_x.keys():
    train_x[l]  =np.vstack([train_x[l], emb])
  else:
    train_x[l] = [emb]

len(train_x.keys())

265

In [58]:
train_l_final = []
label_l_final = []
for k in train_x.keys():
  train_l_final.append(train_x[k])
  label_l_final.append(train.loc[k]['label'])

df_train = pd.DataFrame({'emb': train_l_final, 'label': label_l_final, })
df_train.head()

,emb,label
0,"[[-0.98831576, -0.44439206, 0.99946404, -0.547...",3
1,"[[-0.9883084, -0.44428286, 0.9994648, -0.54771...",7
2,"[[-0.9883366, -0.44480312, 0.9994684, -0.55017...",3
3,"[[-0.9883002, -0.44439593, 0.99946797, -0.5494...",3
4,"[[-0.98991615, -0.44480646, 0.99952805, -0.534...",3


In [59]:
aux = -1
len_l = 0
val_x = {}

for l, emb in zip(val_index_l, val_emb):
  if l in val_x.keys():
    val_x[l]  =np.vstack([val_x[l], emb])
  else:
    val_x[l] = [emb]


val_l_final = []
vlabel_l_final = []
for k in val_x.keys():
  val_l_final.append(val_x[k])
  vlabel_l_final.append(val.loc[k]['label'])

df_val = pd.DataFrame({'emb': val_l_final, 'label': vlabel_l_final})
df_val.head()

,emb,label
0,"[[-0.9895326, -0.39228275, 0.9983325, 0.957225...",3
1,"[[-0.98831064, -0.44451556, 0.99946845, -0.550...",3
2,"[[-0.9883089, -0.44463697, 0.9994666, -0.54908...",5
3,"[[-0.9883187, -0.44453496, 0.99946785, -0.5491...",0
4,"[[-0.9883031, -0.44428778, 0.9994652, -0.54813...",3


In [60]:
df_val, df_test = train_test_split(df_val, test_size=0.4, random_state=35)

In [61]:
from keras import layers
text_input = Input(shape=(None,768,), dtype='float32', name='text')

l_mask = layers.Masking(mask_value=-99.)(text_input)
# Which we encoded in a single vector via a LSTM
encoded_text = layers.LSTM(100,)(l_mask)
out_dense = layers.Dense(30, activation='relu')(encoded_text)
# And we add a softmax classifier on top
out = layers.Dense(len(label_list), activation='softmax')(out_dense)
# At model instantiation, we specify the input and the output:
model = Model(text_input, out)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            (None, None, 768)         0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 768)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               347600    
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                372       
Total params: 351,002
Trainable params: 351,002
Non-trainable params: 0
_________________________________________________________________


In [62]:
df_train.shape, df_val.shape, df_test.shape

((265, 2), (40, 2), (27, 2))

In [63]:
num_sequences = len(df_train['emb'].to_list())
num_sequences

265

In [65]:
num_sequences = len(df_train['emb'].to_list())
batch_size = 5
batches_per_epoch =  53
assert batch_size * batches_per_epoch == num_sequences
num_features= 768
def train_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [66]:
num_sequences_val = len(df_val['emb'].to_list())
num_sequences_val

40

In [67]:
num_sequences_val = len(df_val['emb'].to_list())
batch_size_val = 4
batches_per_epoch_val = 10
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
def val_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size_val][-31:], key=len))
            # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                # print("li", li)
                # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [68]:
from keras.callbacks import ReduceLROnPlateau
call_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

In [69]:
model.fit_generator(train_generator(df_train), steps_per_epoch=batches_per_epoch, epochs=10,
                    validation_data=val_generator(df_val), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )

Epoch 1/10
53/53 [==============================] - 2s 39ms/step - loss: 1.8110 - acc: 0.4415 - val_loss: 1.7709 - val_acc: 0.5500
Epoch 2/10
53/53 [==============================] - 2s 33ms/step - loss: 1.6255 - acc: 0.4679 - val_loss: 1.6116 - val_acc: 0.5500
Epoch 3/10
53/53 [==============================] - 2s 33ms/step - loss: 1.5994 - acc: 0.4679 - val_loss: 1.5563 - val_acc: 0.5500
Epoch 4/10
53/53 [==============================] - 2s 32ms/step - loss: 1.5911 - acc: 0.4679 - val_loss: 1.5271 - val_acc: 0.5500

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
Epoch 5/10
53/53 [==============================] - 2s 33ms/step - loss: 1.5864 - acc: 0.4679 - val_loss: 1.5158 - val_acc: 0.5500
Epoch 6/10
53/53 [==============================] - 2s 33ms/step - loss: 1.5850 - acc: 0.4679 - val_loss: 1.5103 - val_acc: 0.5500
Epoch 7/10
53/53 [==============================] - 2s 33ms/step - loss: 1.5836 - acc: 0.4679 - val_loss: 1.5083 - val_acc: 0.5500

E

In [70]:
num_sequences_val = len(df_test['emb'].to_list())
num_sequences_val

27

In [73]:
num_sequences_val = len(df_test['emb'].to_list())
batch_size_val = 9
batches_per_epoch_val = 3
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
model.evaluate_generator(val_generator(df_test), steps= batches_per_epoch_val)

[1.5186076164245605, 0.5925925970077515]